# Template Mode

The template mode of the uwtools API provides functions that can quickly and easily render Jinja2 templates, with the capability to translate atparse templates so that they can be used with the tool as well. 

For more information, please see the <a href="https://uwtools.readthedocs.io/en/main/sections/user_guide/api/template.html">uwtools.api.template</a> Read the Docs page.

In [1]:
from uwtools.api import template
from pathlib import Path

## render

`template.render` allows for a Jinja2 template to be rendered using the template plus a source of values with which to complete the template.

In [2]:
help(template.render)

Help on function render in module uwtools.api.template:

render(values_src: Union[dict, pathlib.Path, str, NoneType] = None, values_format: Optional[str] = None, input_file: Union[str, pathlib.Path, NoneType] = None, output_file: Union[str, pathlib.Path, NoneType] = None, overrides: Optional[dict[str, str]] = None, env: bool = False, searchpath: Optional[list[str]] = None, values_needed: bool = False, dry_run: bool = False, stdin_ok: bool = False) -> str
    Render a Jinja2 template to a file, based on specified values.

    Primary values used to render the template are taken from the specified file. The format of the
    values source will be deduced from the filename extension, if possible. This can be overridden
    via the ``values_format`` argument. A ``dict`` object may alternatively be provided as the
    primary values source. If no input file is specified, ``stdin`` is read. If no output file is
    specified, ``stdout`` is written to.

    :param values_src: Source of values

Suppose we have a template in the form of a YAML file that looks like the following: 

In [3]:
%%bash
cat fixtures/template/render-template.yaml

user:
  name: {{ first }} {{ last }}
  favorite_food: {{ food }}


The `values_needed` parameter can be used to display which values are needed to complete the template. A logger needs to be initialized for the log of the missing values to be displayed.

In [4]:
import uwtools.logging
uwtools.logging.setup_logging(verbose=False)

print(template.render(input_file='fixtures/template/render-template.yaml', values_needed=True))

[2024-08-18T09:52:37]     INFO Value(s) needed to render this template are:
[2024-08-18T09:52:37]     INFO   first
[2024-08-18T09:52:37]     INFO   food
[2024-08-18T09:52:37]     INFO   last


user:
  name: {{ first }} {{ last }}
  favorite_food: {{ food }}



The log tells us that values will be needed for keys `first`, `food`, and `last`. These values can be sourced from a Python dictionary or from a file. The following file provides the values we need:

In [5]:
%%bash
cat fixtures/template/render-values.yaml

first: John
last: Doe
food: burritos


With these values, we can render the template into a new file with all of the values included. The paths to files containing templates or values can be specified either as a string or a <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object.  If a values file is used to add values to the template, its file type can be specified by `values_format` to a string matching the file type. The resulting rendered file can be specified by setting `output_file` parameter. If `output_file` is not specified or set to `None`, it will be written to `stdout`. 

In [6]:
in_file = 'fixtures/template/render-template.yaml'
values = Path('fixtures/template/render-values.yaml')
out_file = 'fixtures/template/render-complete-1.yaml'

print(template.render(values_src=values, values_format='yaml', input_file=in_file, output_file=out_file))

user:
  name: John Doe
  favorite_food: burritos


It is possible to override values if, for example, some of the values from the values file are wanted, but some are outdated. This can be done by providing a dictionary with the values to override to `overrides`.

In [7]:
new_values = {'first':'Jane', 'food':'tamales'}
new_out_file = 'fixtures/template/render-complete-2.yaml'

print(template.render(values_src=values, values_format='yaml', input_file=in_file, output_file=new_out_file, overrides=new_values))

user:
  name: Jane Doe
  favorite_food: tamales


Let's take a look at the two newly rendered files that we created.

In [8]:
%%bash
cat fixtures/template/render-complete-1.yaml
echo ---------------------------------------
cat fixtures/template/render-complete-2.yaml

user:
  name: John Doe
  favorite_food: burritos
---------------------------------------
user:
  name: Jane Doe
  favorite_food: tamales


## render_to_str

`template.render_to_str` functions similarly to `template.render`, but renders the Jinja2 template to a string. It uses many of the same parameters, but doesn't include the `output_file` parameter since the rendered template won't be written to a file.

In [9]:
help(template.render_to_str)

Help on function render_to_str in module uwtools.api.template:

render_to_str(values_src: Union[dict, pathlib.Path, str, NoneType] = None, values_format: Optional[str] = None, input_file: Union[str, pathlib.Path, NoneType] = None, overrides: Optional[dict[str, str]] = None, env: bool = False, searchpath: Optional[list[str]] = None, values_needed: bool = False, dry_run: bool = False) -> str
    Render a Jinja2 template to a string, based on specified values.

    See ``render()`` for details on arguments, etc.



We can see the resulting string using the same template and values from the first `template.render` example.

In [10]:
result = template.render_to_str(values_src=values, values_format='yaml', input_file=in_file)
print(result)

user:
  name: John Doe
  favorite_food: burritos


For more examples, please refer to the <a href='#render'>render</a> section above.

## translate

This function can be used to translate atparse templates into Jinja2 templates by replacing `@[]` tokens with their corresponding `{{}}` Jinja2 equivalents. 

In [11]:
help(template.translate)

Help on function translate in module uwtools.api.template:

translate(input_file: Union[str, pathlib.Path, NoneType] = None, output_file: Union[str, pathlib.Path, NoneType] = None, dry_run: bool = False, stdin_ok: bool = False) -> bool
    Translate an atparse template to a Jinja2 template.

    ``@[]`` tokens are replaced with Jinja2 ``{{}}`` equivalents. If no input file is specified,
    ``stdin`` is read. If no output file is specified, ``stdout`` is written to. In ``dry_run``
    mode, output is written to ``stderr``.

    :param input_file: Path to atparse file (``None`` => read ``stdin``).
    :param output_file: Path to the file to write the converted template to.
    :param dry_run: Run in dry-run mode?
    :param stdin_ok: OK to read from ``stdin``?
    :return: ``True``.



The template tool has the ability to work with atparse templates like the one shown below.

In [12]:
%%bash
cat 'fixtures/template/translate-template.yaml'

flowers:
  roses: @[ color1 ]
  violets: @[ color2 ]


We can translate this template into a Jinja2 template by providing the path to the atparse template with a string or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object as the `input_file` and the path to the new Jinja2 template as the `output_file`.

In [13]:
in_file = Path('fixtures/template/translate-template.yaml')
out_file = 'fixtures/template/translate-complete.yaml'

template.translate(input_file=in_file, output_file=out_file)

True

Now we have created a Jinja2 template that can be rendered using `template.render` or `template.render_to_str`.

In [14]:
%%bash
cat 'fixtures/template/translate-complete.yaml'

flowers:
  roses: {{  color1  }}
  violets: {{  color2  }}
